<a href="https://colab.research.google.com/github/BenShieh233/Learn_Python/blob/main/AssignmentProblem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Set-Up

In [1]:
#Copy-and-paste the code below to use as "set-up" when your optimization model uses Pyomo. 
#Uncomment the appropriate solver that you need.
#for reference, see https://colab.research.google.com/drive/1yGk8RB5NXrcx9f1Tb-oCiWzbxh61hZLI?usp=sharing

#installing and importing pyomo
!pip install -q pyomo
from pyomo.environ import *

###installing and importing specific solvers (uncomment the one(s) you need)
###glpk
!apt-get install -y -qq glpk-utils
###cbc
#!apt-get install -y -qq coinor-cbc
###ipopt
#!wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
#!unzip -o -q ipopt-linux64
###bonmin
#!wget -N -q "https://ampl.com/dl/open/bonmin/bonmin-linux64.zip"
#!unzip -o -q bonmin-linux64
###couenne
#!wget -N -q "https://ampl.com/dl/open/couenne/couenne-linux64.zip"
#!unzip -o -q couenne-linux64
###geocode
#!wget -N -q "https://ampl.com/dl/open/gecode/gecode-linux64.zip"
#!unzip -o -q gecode-linux64

#Using the solvers:
#SolverFactory('glpk', executable='/usr/bin/glpsol')
#SolverFactory('cbc', executable='/usr/bin/cbc')
#SolverFactory('ipopt', executable='/content/ipopt')
#SolverFactory('bonmin', executable='/content/bonmin')
#SolverFactory('couenne', executable='/content/couenne')
#SolverFactory('gecode', executable='/content/gecode')

     |████████████████████████████████| 9.7 MB 22.0 MB/s 
     |████████████████████████████████| 49 kB 2.6 MB/s 
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 123991 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_4.65-1_amd64.deb ...
Unpacking glpk-utils (4.65-1) ...
Setting up

In [8]:
import pandas as pd

#Code that solves one small assignment problem


In [2]:
#write code that solves one instance of the assignment problem
times = [[193,171,51],
         [189,128,35],
         [39,174,46]]
#first index is the person, second index is the task, times[i][j] denotes the time it takes person i to complete task j

#declare a concrete model
model = ConcreteModel()

num_people = 3 #use i to index the people, this is the first index
num_tasks = 3 #use j to index the tasks, this is the second index

#declare the decision variables
model.x = Var(range(num_people), range(num_tasks), domain=NonNegativeReals) #NOTE: This is how you define 2-dimensional decision variables, model.x[i,j]

#set the objective
model.Objective = Objective(expr = sum(model.x[i,j]*times[i][j] for i in range(num_people) for j in range(num_tasks)), sense = minimize) #this is how you sum up over multiple indices

#set the people constraints
model.PeopleConstraints = ConstraintList() #declaring an empty list of constraints
for i in range(num_people): #add one constraint per person
    model.PeopleConstraints.add(expr = sum(model.x[i,j] for j in range(num_tasks)) <= 1)
    
#set the task constraints
model.TaskConstraints = ConstraintList()
for j in range(num_tasks): #adding one constraint per task
    model.TaskConstraints.add(expr = sum(model.x[i,j] for i in range(num_people)) >= 1)

#model.pprint()


In [3]:
#solve the model
opt = SolverFactory('glpk')
opt.solve(model, tee = False)

{'Problem': [{'Name': 'unknown', 'Lower bound': 218.0, 'Upper bound': 218.0, 'Number of objectives': 1, 'Number of constraints': 7, 'Number of variables': 10, 'Number of nonzeros': 19, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.008649826049804688}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [4]:
#print solution
print("objective:", model.Objective())
for i in range(num_people):
    for j in range(num_tasks):
        if model.x[i,j]() > 0:
            print("person", i, "gets task", j)

objective: 218.0
person 0 gets task 2
person 1 gets task 1
person 2 gets task 0


In [5]:
#alternative solution output
print("objective:", model.Objective())
assignment_matrix = [[model.x[i,j]() for j in range(num_tasks)] for i in range(num_people)]
assignment_matrix

objective: 218.0


[[0.0, 0.0, 1.0], [0.0, 1.0, 0.0], [1.0, 0.0, 0.0]]

In [9]:
#converting assignments to dataframe
pd.DataFrame(assignment_matrix, index = ["person1", "person2", "person3"], columns = ["task1","task2","task3"])

,task1,task2,task3
person1,0.0,0.0,1.0
person2,0.0,1.0,0.0
person3,1.0,0.0,0.0


# Option A

In [13]:
df = pd.read_excel('/content/sample_data/AssignmentProblemData.xlsx', sheet_name = 'Largeproblemdata')
df

,PersonNum,Task1,Task2,Task3,Task4,Task5,Task6,Task7,Task8,Task9,...,Task38,Task39,Task40,Task41,Task42,Task43,Task44,Task45,Task46,Task47
0,1,328,240,99,244,310,83,209,76,66,...,330,134,182,84,341,193,327,54,160,312
1,2,153,111,96,308,297,114,51,178,167,...,114,59,336,91,308,104,253,199,62,118
2,3,82,246,163,277,68,55,82,232,313,...,86,113,95,151,160,185,96,320,232,254
3,4,149,171,81,172,163,298,319,100,155,...,101,337,222,161,192,207,190,292,53,133
4,5,201,211,224,323,133,112,165,186,116,...,144,195,307,253,280,237,232,277,204,130
5,6,299,254,233,107,291,244,254,229,200,...,80,311,144,76,213,248,98,243,148,167
6,7,265,329,239,202,126,247,75,324,258,...,289,303,298,149,174,133,98,256,235,99
7,8,282,183,255,276,271,249,177,266,267,...,97,177,211,74,114,262,109,146,89,147
8,9,125,73,225,221,208,327,322,213,231,...,148,120,315,141,101,135,64,215,161,280
9,10,180,112,58,291,95,202,262,161,124,...,312,349,264,214,249,70,221,331,248,331


In [19]:
#write code that solves one instance of the assignment problem
times2 = df.iloc[:, 1:].values.tolist()

In [21]:
#declare a concrete model
model2 = ConcreteModel()

num_people2 = len(df)
num_tasks2 = len(df.iloc[0,1:])

#declare the decision variables
model2.x = Var(range(num_people2), range(num_tasks2), domain=NonNegativeReals)

#set the objective
model2.Objective = Objective(expr = sum(model2.x[i,j]*times2[i][j] for i in range(num_people2) for j in range(num_tasks2)), sense = minimize)

#set the people constraints
model2.PeopleConstraints = ConstraintList() #declaring an empty list of constraints
for i in range(num_people2): #add one constraint per person
    model2.PeopleConstraints.add(expr = sum(model2.x[i,j] for j in range(num_tasks2)) <= 1)

#set the task constraints
model2.TaskConstraints = ConstraintList()
for j in range(num_tasks2): #adding one constraint per task
    model2.TaskConstraints.add(expr = sum(model2.x[i,j] for i in range(num_people2)) >= 1)

model.pprint()

5 Set Declarations
    PeopleConstraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    TaskConstraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    x_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain              : Size : Members
        None :     2 : x_index_0*x_index_1 :    9 : {(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)}
    x_index_0 : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}
    x_index_1 : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

1 Var Declarations
    x : Size=9, Index=x_index
        Key    : Lower : Value : Upper : Fixed : Stale : Domain
        (

In [24]:
#solve the model
opt = SolverFactory('glpk')
opt.solve(model2, tee = False)

{'Problem': [{'Name': 'unknown', 'Lower bound': 2815.0, 'Upper bound': 2815.0, 'Number of objectives': 1, 'Number of constraints': 95, 'Number of variables': 2210, 'Number of nonzeros': 4419, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.025246858596801758}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [25]:
#print solution
print("objective:", model2.Objective())
for i in range(num_people2):
    for j in range(num_tasks2):
        if model2.x[i,j]() > 0:
            print("person", i, "gets task", j)

objective: 2815.0
person 0 gets task 8
person 1 gets task 6
person 2 gets task 5
person 3 gets task 45
person 4 gets task 35
person 5 gets task 22
person 6 gets task 33
person 7 gets task 34
person 8 gets task 12
person 9 gets task 42
person 10 gets task 39
person 11 gets task 23
person 12 gets task 11
person 13 gets task 2
person 14 gets task 36
person 15 gets task 10
person 16 gets task 31
person 17 gets task 25
person 18 gets task 41
person 19 gets task 7
person 20 gets task 20
person 21 gets task 4
person 22 gets task 15
person 23 gets task 43
person 24 gets task 19
person 25 gets task 14
person 26 gets task 9
person 27 gets task 18
person 28 gets task 28
person 29 gets task 21
person 30 gets task 3
person 31 gets task 29
person 32 gets task 46
person 33 gets task 30
person 34 gets task 44
person 35 gets task 37
person 36 gets task 40
person 37 gets task 13
person 38 gets task 24
person 39 gets task 0
person 40 gets task 27
person 41 gets task 17
person 42 gets task 16
person 43 ge

# Option B

In [27]:
#load in data (located on the first sheet)
df2 = pd.read_excel('/content/sample_data/AssignmentProblemData.xlsx', sheet_name='50problemsdata') 
df2

,ProblemNum,Person1Task1,Person1Task2,Person1Task3,Person2Task1,Person2Task2,Person3Task3,Person3Task1,Person3Task2,Person3Task3.1
0,1,193,171,51,189,128,35,39,174,46
1,2,117,19,37,53,32,92,189,40,185
2,3,144,126,136,68,118,32,43,60,193
3,4,12,100,149,130,72,52,116,188,107
4,5,13,147,188,186,27,68,102,125,28
5,6,103,51,190,188,82,14,38,111,181
6,7,82,193,49,56,112,130,88,181,181
7,8,133,174,46,105,107,26,195,42,156
8,9,114,18,108,138,183,178,193,190,14
9,10,72,178,85,42,61,95,155,20,154


In [66]:
num_people = 3
num_tasks = 3 
def get_solutions(x):
  times=[]
  for i in range(0, len(df2.iloc[x,1:]),num_people):
      times.append(df2.iloc[x,i+1:i+4].values.tolist())
  model = ConcreteModel()
  model.x = Var(range(num_people), range(num_tasks), domain=NonNegativeReals)
  model.Objective = Objective(expr = sum(model.x[i,j]*times[i][j] for i in range(num_people) for j in range(num_tasks)), sense = minimize)
  model.PeopleConstraints = ConstraintList() #declaring an empty list of constraints
  for i in range(num_people): #add one constraint per person
      model.PeopleConstraints.add(expr = sum(model.x[i,j] for j in range(num_tasks)) <= 1)
  model.TaskConstraints = ConstraintList()
  for j in range(num_tasks): #adding one constraint per task
      model.TaskConstraints.add(expr = sum(model.x[i,j] for i in range(num_people)) >= 1)
  opt = SolverFactory('glpk')
  opt.solve(model, tee = False)
  print("objective:", model.Objective())
  assignment_matrix = [[model.x[i,j]() for j in range(num_tasks)] for i in range(num_people)]
  assignment = pd.DataFrame(assignment_matrix, index = ["person1", "person2", "person3"], columns = ["task1","task2","task3"])
  return assignment.to_dict()

In [67]:
get_solutions(0)

objective: 218.0


{'task1': {'person1': 0.0, 'person2': 0.0, 'person3': 1.0},
 'task2': {'person1': 0.0, 'person2': 1.0, 'person3': 0.0},
 'task3': {'person1': 1.0, 'person2': 0.0, 'person3': 0.0}}

In [68]:
solutions = []
for i in range(len(df2)):
  solutions.append(get_solutions(i))

objective: 218.0
objective: 130.0
objective: 201.0
objective: 191.0
objective: 68.0
objective: 103.0
objective: 249.0
objective: 193.0
objective: 170.0
objective: 147.0
objective: 79.0
objective: 266.0
objective: 229.0
objective: 140.0
objective: 106.0
objective: 205.0
objective: 296.0
objective: 149.0
objective: 204.0
objective: 196.0
objective: 291.0
objective: 221.0
objective: 171.0
objective: 247.0
objective: 294.0
objective: 152.0
objective: 159.0
objective: 170.0
objective: 125.0
objective: 131.0
objective: 296.0
objective: 232.0
objective: 301.0
objective: 338.0
objective: 167.0
objective: 124.0
objective: 163.0
objective: 181.0
objective: 297.0
objective: 174.0
objective: 158.0
objective: 119.0
objective: 165.0
objective: 288.0
objective: 121.0
objective: 174.0
objective: 148.0
objective: 248.0
objective: 90.0
objective: 195.0


In [69]:
df2['solution'] = solutions
df2

,ProblemNum,Person1Task1,Person1Task2,Person1Task3,Person2Task1,Person2Task2,Person3Task3,Person3Task1,Person3Task2,Person3Task3.1,solution
0,1,193,171,51,189,128,35,39,174,46,"{'task1': {'person1': 0.0, 'person2': 0.0, 'pe..."
1,2,117,19,37,53,32,92,189,40,185,"{'task1': {'person1': 0.0, 'person2': 1.0, 'pe..."
2,3,144,126,136,68,118,32,43,60,193,"{'task1': {'person1': 0.0, 'person2': 0.0, 'pe..."
3,4,12,100,149,130,72,52,116,188,107,"{'task1': {'person1': 1.0, 'person2': 0.0, 'pe..."
4,5,13,147,188,186,27,68,102,125,28,"{'task1': {'person1': 1.0, 'person2': 0.0, 'pe..."
5,6,103,51,190,188,82,14,38,111,181,"{'task1': {'person1': 0.0, 'person2': 0.0, 'pe..."
6,7,82,193,49,56,112,130,88,181,181,"{'task1': {'person1': 0.0, 'person2': 0.0, 'pe..."
7,8,133,174,46,105,107,26,195,42,156,"{'task1': {'person1': 0.0, 'person2': 1.0, 'pe..."
8,9,114,18,108,138,183,178,193,190,14,"{'task1': {'person1': 0.0, 'person2': 1.0, 'pe..."
9,10,72,178,85,42,61,95,155,20,154,"{'task1': {'person1': 0.0, 'person2': 1.0, 'pe..."
